This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2026.1_tutorials/wallaroo-model-operations-tutorials/deploy/by-framework/onnx/imdb).

## IMDB Sentiment Model Deployment Tutorial

The following example demonstrates how to use Wallaroo with chained models.  In this example, we will be using information from the IMDB (Internet Movie DataBase) with a sentiment model to detect whether a given review is positive or negative.  Imagine using this to automatically scan Tweets regarding your product and finding either customers who need help or have nice things to say about your product.

Note that this example is considered a "toy" model - only the first 100 words in the review were tokenized, and the embedding is very small.

The following example is based on the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/), and sample data can be downloaded from the [aclIMDB dataset](http://s3.amazonaws.com/text-datasets/aclImdb.zip ).

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed:
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * [`pyarrow`](https://pypi.org/project/pyarrow/): PyArrow for Apache Arrow support
  * [`polars`](https://pypi.org/project/polars/): Polars for DataFrame with native Apache Arrow support

In [13]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# to display dataframe tables
from IPython.display import display
# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import pyarrow as pa
import time

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

To test this model, we will perform the following:

* Create a workspace for our models.
* Upload two models:
  * `embedder`: Takes pre-tokenized text documents (model input: 100 integers/datum; output 800 numbers/datum) and creates an embedding from them.
  * `sentiment`:  The second model classifies the resulting embeddings from 0 to 1, which 0 being an unfavorable review, 1 being a favorable review.
* Create a pipeline that will take incoming data and pass it to the embedder, which will pass the output to the sentiment model, and then export the final result.
* To test it, we will use information that has already been tokenized and submit it to our pipeline and gauge the results.

Just for the sake of this tutorial, we'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

When we create our new workspace, we'll save it in the Python variable `workspace` so we can refer to it as needed.

First we'll create a workspace for our environment, and call it `imdbworkspace`.  We'll also set up our pipeline so it's ready for our models.

In [3]:
workspace_name = f'imdbworkspace'
pipeline_name = f'imdbpipeline'

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

imdb_pipeline = wl.build_pipeline(pipeline_name)
imdb_pipeline

name,imdbpipeline
created,2025-05-01 18:49:25.343129+00:00
last_updated,2025-05-01 18:49:25.343129+00:00
deployed,(none)
workspace_id,15
workspace_name,imdbworkspace
arch,None
accel,None
tags,
versions,28f3c518-404d-4315-9d09-b981d11c39af
steps,


Just to make sure, let's list our current workspace.  If everything is going right, it will show us we're in the `imdb-workspace`.

In [5]:
wl.get_current_workspace()

{'name': 'imdbworkspace', 'id': 15, 'archived': False, 'created_by': 'ea2c3caf-86e1-4501-ad82-c8a09d518bf1', 'created_at': '2025-05-01T18:49:25.199976+00:00', 'models': [], 'pipelines': [{'name': 'imdbpipeline', 'create_time': datetime.datetime(2025, 5, 1, 18, 49, 25, 343129, tzinfo=tzutc()), 'definition': '[]'}]}

Now we'll upload our two models:

* `embedder.onnx`: This will be used to embed the tokenized documents for evaluation.
* `sentiment_model.onnx`: This will be used to analyze the review and determine if it is a positive or negative review.  The closer to 0, the more likely it is a negative review, while the closer to 1 the more likely it is to be a positive review.

In [6]:
embedder = (wl.upload_model(f'embedder-o', 
                            './embedder.onnx', 
                            framework=wallaroo.framework.Framework.ONNX)
                            .configure(tensor_fields=["tensor"])
            )
smodel = (wl.upload_model(f'smodel-o', 
                          './sentiment_model.onnx', 
                          framework=wallaroo.framework.Framework.ONNX)
                          .configure(runtime="onnx", tensor_fields=["flatten_1"])
        )

With our models uploaded, now we'll create our pipeline that will contain two steps:

* First, it runs the data through the embedder.
* Second, it applies it to our sentiment model.

In [7]:
# now make a pipeline
imdb_pipeline.add_model_step(embedder)
imdb_pipeline.add_model_step(smodel)

name,imdbpipeline
created,2025-05-01 18:49:25.343129+00:00
last_updated,2025-05-01 18:49:25.343129+00:00
deployed,(none)
workspace_id,15
workspace_name,imdbworkspace
arch,None
accel,None
tags,
versions,28f3c518-404d-4315-9d09-b981d11c39af
steps,


Now that we have our pipeline set up with the steps, we can deploy the pipeline.

In [15]:
imdb_pipeline.deploy(wait_for_status=False)

Deployment initiated for imdbpipeline. Please check pipeline status.


name,imdbpipeline
created,2025-05-01 18:49:25.343129+00:00
last_updated,2025-05-01 18:53:48.122694+00:00
deployed,True
workspace_id,15
workspace_name,imdbworkspace
arch,x86
accel,none
tags,
versions,"abdfcaf7-c400-491a-a8e7-bf933e93eb70, 40030502-d008-4a70-b76b-cb2607ebf01f, b51772e6-a001-4889-b4b2-dc4d144c03fa, 28f3c518-404d-4315-9d09-b981d11c39af"
steps,embedder-o


We'll check the pipeline status to verify it's deployed and the models are ready.

In [16]:
time.sleep(15)

while imdb_pipeline.status()['status'] != 'Running':
    time.sleep(15)
    print("Waiting for deployment.")
    imdb_pipeline.status()['status']
imdb_pipeline.status()['status']


'Running'

To test this out, we'll start with a single piece of information from our data directory.

In [17]:
singleton = pd.DataFrame.from_records(
    [
    {
        "tensor":[
            1607.0,
            2635.0,
            5749.0,
            199.0,
            49.0,
            351.0,
            16.0,
            2919.0,
            159.0,
            5092.0,
            2457.0,
            8.0,
            11.0,
            1252.0,
            507.0,
            42.0,
            287.0,
            316.0,
            15.0,
            65.0,
            136.0,
            2.0,
            133.0,
            16.0,
            4311.0,
            131.0,
            286.0,
            153.0,
            5.0,
            2826.0,
            175.0,
            54.0,
            548.0,
            48.0,
            1.0,
            17.0,
            9.0,
            183.0,
            1.0,
            111.0,
            15.0,
            1.0,
            17.0,
            284.0,
            982.0,
            18.0,
            28.0,
            211.0,
            1.0,
            1382.0,
            8.0,
            146.0,
            1.0,
            19.0,
            12.0,
            9.0,
            13.0,
            21.0,
            1898.0,
            122.0,
            14.0,
            70.0,
            14.0,
            9.0,
            97.0,
            25.0,
            74.0,
            1.0,
            189.0,
            12.0,
            9.0,
            6.0,
            31.0,
            3.0,
            244.0,
            2497.0,
            3659.0,
            2.0,
            665.0,
            2497.0,
            63.0,
            180.0,
            1.0,
            17.0,
            6.0,
            287.0,
            3.0,
            646.0,
            44.0,
            15.0,
            161.0,
            50.0,
            71.0,
            438.0,
            351.0,
            31.0,
            5749.0,
            2.0,
            0.0,
            0.0
        ]
    }
]
)
results = imdb_pipeline.infer(singleton)
display(results)

,time,in.tensor,out.dense_1,anomaly.count
0,2025-05-01 18:54:07.076,"[1607.0, 2635.0, 5749.0, 199.0, 49.0, 351.0, 16.0, 2919.0, 159.0, 5092.0, 2457.0, 8.0, 11.0, 1252.0, 507.0, 42.0, 287.0, 316.0, 15.0, 65.0, 136.0, 2.0, 133.0, 16.0, 4311.0, 131.0, 286.0, 153.0, 5.0, 2826.0, 175.0, 54.0, 548.0, 48.0, 1.0, 17.0, 9.0, 183.0, 1.0, 111.0, 15.0, 1.0, 17.0, 284.0, 982.0, 18.0, 28.0, 211.0, 1.0, 1382.0, 8.0, 146.0, 1.0, 19.0, 12.0, 9.0, 13.0, 21.0, 1898.0, 122.0, 14.0, 70.0, 14.0, 9.0, 97.0, 25.0, 74.0, 1.0, 189.0, 12.0, 9.0, 6.0, 31.0, 3.0, 244.0, 2497.0, 3659.0, 2.0, 665.0, 2497.0, 63.0, 180.0, 1.0, 17.0, 6.0, 287.0, 3.0, 646.0, 44.0, 15.0, 161.0, 50.0, 71.0, 438.0, 351.0, 31.0, 5749.0, 2.0, 0.0, 0.0]",[0.37142318],0


Since that works, let's load up all 50,000 rows and do a full inference on each of them via an Apache Arrow file.  Wallaroo pipeline inferences use Apache Arrow as their core data type, making this inference fast.  

We'll do a demonstration with a pandas DataFrame and display the first 5 results.

In [18]:
results = imdb_pipeline.infer_from_file('./data/test_data_50K.arrow')

In [19]:
# using pandas DataFrame

outputs = results.to_pandas()
display(outputs.loc[:5, ["time","out.dense_1"]])

,time,out.dense_1
0,2025-05-01 18:54:09.444,[0.8980188]
1,2025-05-01 18:54:09.444,[0.056596935]
2,2025-05-01 18:54:09.444,[0.9260802]
3,2025-05-01 18:54:09.444,[0.926919]
4,2025-05-01 18:54:09.444,[0.6618577]
5,2025-05-01 18:54:09.444,[0.48736304]


## Undeploy

With our pipeline's work done, we'll undeploy it and give our Kubernetes environment back its resources.

In [20]:
imdb_pipeline.undeploy()

name,imdbpipeline
created,2025-05-01 18:49:25.343129+00:00
last_updated,2025-05-01 18:53:48.122694+00:00
deployed,False
workspace_id,15
workspace_name,imdbworkspace
arch,x86
accel,none
tags,
versions,"abdfcaf7-c400-491a-a8e7-bf933e93eb70, 40030502-d008-4a70-b76b-cb2607ebf01f, b51772e6-a001-4889-b4b2-dc4d144c03fa, 28f3c518-404d-4315-9d09-b981d11c39af"
steps,embedder-o


And there is our example. Please feel free to contact us at Wallaroo for if you have any questions.